# MinHash & LSH 
Okay, so I think I will understand this better if I implement the various pieces of the algorithm/s in python first, and then figure out how they will work in MyriaL. 

From p. 91 of the MMDS book, we have these steps for combining min-hash and LSH. I switched the book's variables to the ones we have been using, specifically:

    n: as in n-gram
    m: # of groups of k hash fns
    k: # of hash functions in a group
    l: length of minhash signatures
    t: threshold

1. Pick a value of n and construct from each document the set of n-shingles.
2. Sort the document-shingle pairs to order them by shingle.
3. Pick a length l for the minhash signatures. Feed the sorted list to the algorithm of Section 3.3.5 to compute the minhash signatures for all the documents.
4. Choose a threshold t that defines how similar documents have to be in order for them to be regarded as a desired “similar pair.” Pick a number of bands m and a number of rows k such that mk = l, and the threshold t is approximately (1/m)1/k. If avoidance of false negatives is important, you may wish to select m and r to produce a threshold lower than t; if speed is important and you wish to limit false positives, select m and k to produce a higher threshold.
5. Construct candidate pairs by applying the LSH technique of Section 3.4.1.
6. Examine each candidate pair’s signatures and determine whether the fraction of components in which they agree is at least t.
7. Optionally, if the signatures are sufficiently similar, go to the documents themselves and check that they are truly similar, rather than documents that, by luck, had similar signatures.

I will break this computation down by the steps above.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline 

In [2]:
# Load the cora data
cora_data = pd.DataFrame.from_csv("../data/cora/cora.txt", sep='\t', header=None)

# 1 Pick a value of n and compute n-grams
`n=5` for now

According to the MMDS book, `n=5` should work, except common letters skew this for large documents, and `n=9` is considered safe for large documents

In [3]:
# from http://locallyoptimal.com/blog/2013/01/20/elegant-n-gram-generation-in-python/
def find_ngrams(input_list, n):
  return zip(*[input_list[i:] for i in range(n)])

In [4]:
# Compute all n-grams for the cora data
cora_data.fillna("", inplace=True)
cora_data['ngram_author'] = cora_data[2].apply(find_ngrams, n=5)
cora_data['ngram_volume'] = cora_data[3].apply(find_ngrams, n=5)
cora_data['ngram_title'] = cora_data[4].apply(find_ngrams, n=5)
cora_data['ngram_institute'] = cora_data[5].apply(find_ngrams, n=5)
cora_data['ngram_venue'] = cora_data[6].apply(find_ngrams, n=5)
cora_data['ngram_address'] = cora_data[7].apply(find_ngrams, n=5)
cora_data['ngram_pub'] = cora_data[8].apply(find_ngrams, n=5)
cora_data['ngram_year'] = cora_data[9].apply(find_ngrams, n=5)
cora_data['ngram_pages'] = cora_data[10].apply(find_ngrams, n=5)
cora_data['ngram_editor'] = cora_data[11].apply(find_ngrams, n=5)
cora_data['ngram_note'] = cora_data[12].apply(find_ngrams, n=5)
cora_data['ngram_month'] = cora_data[13].apply(find_ngrams, n=5)

In [5]:
# Append all the n-grams of the cora data
cora_ngrams = []
for record in cora_data.ix[:,14:].values.tolist():
    cora_ngrams.append([item for sublist in record for item in sublist])

# 2. Order the records by ngram
## Is this the correct interpretation? Probably not...
"Sort the document-shingle pairs to order them by shingle." --> Order the records by ngram

Do we need to sort the characters within the ngrams?



In [7]:
cora_ngrams_sorted = sorted(cora_ngrams)

# 3. Pick a length l for minhash signatures & compute minhash

Oooookay this is a big step.

`l` is the number of times we permute the rows (records?), so I guess `l` should approximately equal the alphabet size? This intuition could be way off. If we include lower case letters, upper case letters, digits, punctuation, & special characters, there are 95 ascii characters.

using `l=95`


Resources: 

* Section 3.3.5 in MMDS

* https://rajmak.wordpress.com/2014/12/22/locality-sensitive-hashing-lsh-map-reduce-in-python/

* Copied from the book from Dan p.1115 (variables not changed to what we've been using):

      FOR i := 1 TO k DO
          FOR j := 1 TO m DO
              Vij := inf;
      FOR EACH basket b DO BEGIN
          FOR j := 1 TO m DO 
              compute hj(b);
          FOR EACH item i in b DO 
              FOR j := 1 TO m DO
                  IF hj(b)<Vij THEN Vij := hj(b);
      END
    


# 4. Choose a threshold, pick k & m accordingly

# 5. Apply LSH to pick candidate pairs

# 6. Compute P(A=B) from the minhash signature
"Examine each candidate pair’s signatures and determine whether the fraction of components in which they agree is at least t." --> Is this effectively computing P(A=B) from the minhash signature?

# 7. Compute J(A, B) on the pairs that pass P(A=B)>t if we care about false positives